Definitely failed:
- PALSAR raster
- nDOM (no noticeable performance improvement and introduces unnecessary complexity)
- similarity matrix (as it is just high level and very unreadable for all bands and indices)

Hurdles:
- Limits of Google Earth Engine
- Slowness of computations, especially near the end, when everything had to work together
- keep oneself organized when having to manage everything within one project

Still interesting
- base area prediction

# Similarity Matrix and Dendrogram

In [ ]:
# List the optimal bands
from ltm.models import bands_from_importance

sentinel_bands, indices = bands_from_importance(
    "../reports/band_importance.csv"
)

In [ ]:
from ltm.features import get_similarity_matrix, load_raster, show_similarity_matrix
from ltm.data import split_band_name

data = load_raster("../data/processed/band_importance/data.tif")

bands = sentinel_bands + indices
columns = [column for column in data.columns if split_band_name(column)[2].replace(" ", "_") in bands]

In [ ]:
similarity_matrix = get_similarity_matrix(data[columns], method="pearson")
show_similarity_matrix(similarity_matrix)

In [ ]:
from ltm.features import show_dendrogram

show_dendrogram(similarity_matrix)

# Show timeseries

In [ ]:
from ltm.data import show_timeseries, sentinel_composite
from pathlib import Path
from datetime import datetime

data_path = "../data/processed/fails/timeseries.tif"
target_path = "../data/processed/target.tif"

rgb_bands = ["B4", "B3", "B2"]

if not Path(data_path).exists():
    Path(data_path).parent.mkdir(parents=True, exist_ok=True)

    sentinel_composite(
        target_path_from=target_path,
        data_path_to=data_path,
        time_window=(datetime(2017, 4, 1), datetime(2018, 4, 1)),
        num_composites=12,
        sentinel_bands=rgb_bands,
    )

show_timeseries(data_path, "Mean", rgb_bands)